General imports that we'll need
I highly reccomend creating a virtual enviroment for all Polymarket related items as some aspect of the client require specefic versions of tools.

In [1]:
from py_clob_client.client import ClobClient
from py_clob_client.clob_types import OrderArgs, PartialCreateOrderOptions, MarketOrderArgs, OrderScoringParams, OpenOrderParams
from py_clob_client.order_builder.constants import BUY, SELL
from py_clob_client.exceptions import PolyApiException
from math import floor

Now lets set up the variables we'll need to initialize the client. 

Here are what the following variables represent:
funder: The Polymarket address associated with the browser wallet login OR the email login.
host: the Polymarket CLOB endpoint https://clob.polymarket.com
key: The private key exported from Magic that is associated with the account OR The private key exported from the Browser Wallet that is associated with the account
    If you use the email login you need to visit https://reveal.magic.link/
    If you use metamask/web3 login you need to export the private key from that application. 
chain_id: For the majority of use cases this can be safely set to 137(Polygon Mainnet)
signature_type: Again, generally this will be 1



In [2]:
funder = "" #The Funder is the adress as seen on your Polymarket profile.
host = "https://clob.polymarket.com"
key = "" #This is your private key. See above for details 
chain_id = 137
signature_type = 1

In [3]:
client = ClobClient(
host,               # the polymarket host
key=key,            # your private key exported from magic/polymarket
chain_id=chain_id,  # 137
signature_type=1,   # this type of config requires this signature type
funder=funder) # this is your polymarket public address (where you send the usdc)


In the documentation you'll find the ability to export, save or generate API creds. This is generally unnecesary but nice to have. The below code handles creating new keys if neccesary or using existing ones if keys do exist. 

In [4]:
client.set_api_creds(client.create_or_derive_api_creds())


In order to follow DRY(DON'T REPEAT YOURSELF) principles lets create a simple function that can handle Buying and Selling for us. 
The function takes an action "BUY" or "SELL", a TOKENId and the number of shares to buy. To find a token ID please see the "Market Info" Notebook. 

In [10]:
#In production you absolutely need try/catch blocks here but for the purposes of simplicity we're ignoring that neccesity for now. 
def buy_sell_limit_order(action: str, tokenID: str, numShares: int, price: float = .99, neg_risk_status: bool = False):
    if action == 'BUY':
        order_args = OrderArgs(
            token_id=tokenID,
            side='BUY',
            size=floor(numShares),
            price=price
            )
        signed_order = client.create_order(order_args, PartialCreateOrderOptions(neg_risk=neg_risk_status))
    elif action == 'SELL':
        order_args = OrderArgs(
            token_id=tokenID,
            side='SELL',
            price=price,#hardcode for sell
            size=numShares #will be shares in this case
                        )
        signed_order = client.create_order(order_args, PartialCreateOrderOptions(neg_risk=neg_risk_status))
    else: 
        print(f"Invalid action. Action recieved: {action}")
        return


    #NotEnough balance/allowance can mean that you dont have enough shares 
    #invalid sig can mean its negrisk or your funder is 
    resp = client.post_order(signed_order)
    print(f"Order Response: {resp}")


Couple notes on the above function. These are limit orders so you'll need to specify the price you're willing to buy or sell at and the number of shares to buy/sell at theat price. You also need to specify rather a market is negative risk or not. Check the docs for details on that but in basic terms if there are multiple mutually exclusive outcomes(2+) the market is neg_risk. 

In [14]:
buy_sell_limit_order('BUY',"55339638397443112919791846485253629869798811464400272253569648598507361954673",5,price=.01,neg_risk_status=False)

Order Response: {'errorMsg': '', 'orderID': '0xb234bd04fe355e6f0579069434b1ff0f172121cccff13f569f3e01d3318faa8a', 'takingAmount': '', 'makingAmount': '', 'status': 'live', 'transactionsHashes': None, 'success': True}
